# Test to pass the SWMM model values of DWF to the WEST model 

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import swmmio
import numpy as np
import math, os
from itertools import chain
from pyswmm import Simulation, Links, Output
from swmm.toolkit.shared_enum import LinkAttribute
from datetime import datetime, timedelta


import xml.etree.ElementTree as ET
import re

import SWMM_InpConstants as SWWM_C


import findPaths as fp

#hydraulic constants
VISCO = 1.0035*(10**-6)
GRAVITY = 9.81



#created for the algorithm
SLOPE = 'Slope'
VMAX = 'Vmax'
K = 'k'
TANK_INDEXES = 'tanksIndexes'
END = 'EndNode'
N_PEOPLE = 'NumberPeople'
TIMEPATTERN ='TimePattern'
TIMESERIES = 'TimeSeries'
AGG_CATCHMENT = 'AggregateAtNearestCatchment'

LINKS = "Links"
LEAVES = "Leaves"
SUBCATCHMENTS = "Catchments"
DIRECTF = "DirectWaterFlows"
T_PATTERNS = "Patterns"
DWFS = "DWFs"

DIST_TO_LEAF = "DistanceToOriginLeaf"
BREAK_POINT = "BreakPoint"
PATH_ELEMENTS = "ElementsOnThePath"
PATH_ELEMENTS_INI = "ElementsOnTheBegginingOfThePath"
MODELED_INPUT = "InputModelledInSWMM"
QPERPERSON_LABEL = "FlowPerPerson"


In [2]:
#Values that can be changed 

WTP_COL = "DOM_1631424" 
STSACRA_TO_LIMOLIOU_COL = "U002_Pompe_IREU" #Pump
PASCAL_COL = "U004" #Pump

#Names of the measurement file
ESTA_ME = "Station Est (m³/h)"
PASCAL_ME ='U004 St-Pascal (m³/h)'


#MODEL points for checking the flow
LIMOILOU_COL="DOM_941798"
PASCAL_COL = "U004"
STSACRA_COL = "UNI_602608"
BEAU_COL = 'Pompe-Beauport'
NOEST_COL= 'U003_Nord_Ouest'
LINKS_MEASS_FLOW = [LIMOILOU_COL,BEAU_COL,STSACRA_COL,NOEST_COL,PASCAL_COL]

WTP_Tank = "RA_606859"
ST_SACR_LEAF_NODE= "R007637"
BEAUPORT_LEAF_NODE= "J79"

FLOW_PER_PERSON = 0.4  #m3/d as in WEST. In SWMM units are m3/s 



In [3]:

#ONLY WORKS IF THERE ARE NOT PATTERNS DIFFERENT FROM HOURLY!!! TODO
#returns a dictionary with the inp time patterns
def getHourlyPatterns(networkFile):
    
    #Gets the section from the inp
    patternsRaw = swmmio.utils.dataframes.dataframe_from_inp(networkFile,'PATTERNS')
    #Gets the names of the patterns
    hourlyPatternsNames = patternsRaw[SWWM_C.PATTERNS].str.extract(r'^(.*?)\sHOURLY').iloc[:,0].unique()
    hPatterns = [value.strip() for value in hourlyPatternsNames if isinstance(value, str)]
    #Removes the HOURLY word and expands the values in different columns
    patternsRaw[SWWM_C.PATTERNS] = patternsRaw[SWWM_C.PATTERNS].str.replace(r'\sHOURLY\b', '', regex=True).str.strip()
    hourlyPatterns = patternsRaw[SWWM_C.PATTERNS].str.split(r'\s+', expand=True).set_index(0)
    
    #Creates the dicitonary
    patterns = {}
    for hp in hPatterns:
        df = hourlyPatterns.loc[hp] #selects all the rows refering to the pattern
        vals = list(chain.from_iterable(df.values.tolist())) #it flattens the values on a list
        patterns[hp] = vals
    
    return patterns

def getsNetwork(networkFile):
    
    nElements = {}
    
    # Instantiate a swmmio model object
    model = swmmio.Model(networkFile)

    # Pandas dataframes with nodes and links
    #DONT USE CONDUITS, they are more but dont have all the links (bug of the library, or inp?)
    fileName, extension = os.path.splitext(networkFile)
    outfile = f"{fileName}.out"
    if not os.path.exists(outfile):
        raise FileNotFoundError(f"Output file '{outfile}' does not exist. Please run a simulation in SWMM before.")

    links = model.links()[[SWWM_C.IN_NODE,SWWM_C.OUT_NODE,SWWM_C.LEN,SWWM_C.DIAM,
                           SWWM_C.MAX_Q,SWWM_C.ROUG,SWWM_C.INOFF,SWWM_C.OUTOFF,SWWM_C.SHAPE]].copy()
    nodes = model.nodes().index.to_series()
    subCatch = model.subcatchments()[[SWWM_C.CATCH_OUT,SWWM_C.AREA]]
    dwfs = model.inp.dwf[[SWWM_C.INFLOW_TYPE,SWWM_C.INFLOW_MEAN,SWWM_C.INFLOW_PATTERNS]].copy()
    #Only flow DWFS !! 
    dwfsWater = dwfs[dwfs[SWWM_C.INFLOW_TYPE]==SWWM_C.FLOW].drop(columns=[SWWM_C.INFLOW_TYPE]).copy()
    directFlows = model.inp.inflows #can't handle direct inflows with a baseline pattern !!! IMPORTANT TODO
    #Only direct water flows not other constituents
    directWaterFlows = directFlows[(directFlows[SWWM_C.DFLOW_CONSTITUENT]==SWWM_C.FLOW)&(directFlows[SWWM_C.TYPE]==SWWM_C.FLOW)].drop(columns=[SWWM_C.DFLOW_CONSTITUENT,SWWM_C.TYPE,SWWM_C.DFLOW_MFACTOR])
    #Only the time series related to direct water flows
    timeS = model.inp.timeseries.loc[directWaterFlows[SWWM_C.DFLOW_TIMES].dropna().unique().tolist()]
    
    patterns = getHourlyPatterns(networkFile) #returns a dicitonary 
    
    #remove subtachments with area 0
    totalSubcathments = subCatch.shape[0]
    subCatch = subCatch[subCatch[SWWM_C.AREA]>0]
    print("There were ", totalSubcathments - subCatch.shape[0], "subcatchments with area 0")
    
    #check that the network uses elevation
    assert model.inp.options.loc['LINK_OFFSETS'][0] == SWWM_C.ELEVATION

    #Get outlet and inlet nodes of pipes and cathments
    outNodes = links[SWWM_C.OUT_NODE].drop_duplicates()
    inNodes = links[SWWM_C.IN_NODE].drop_duplicates()

    #Get only nodes that are not outlets 
    startPoints = nodes[~nodes.isin(outNodes.tolist())]
    #Removes nodes that are not connected to the network
    sPointsCleaned = startPoints[startPoints.isin(inNodes.tolist())]
    
    nElements[LINKS] = links
    nElements[LEAVES] = sPointsCleaned
    nElements[SUBCATCHMENTS] = subCatch
    nElements[DWFS] = dwfsWater
    nElements[DIRECTF] = directWaterFlows
    nElements[T_PATTERNS] = patterns # Not being used at the moment
    nElements[TIMESERIES] = timeS
    
    print("Number of nodes ", nodes.shape[0],", outlets ", outNodes.shape[0], ", startPoints ",startPoints.shape[0], ", and after cleaned ",sPointsCleaned.shape[0])

    return nElements, outfile


#assigns the properties to the sewer section 
def createSewerWEST(group,shapeType,tankIndex):
    
    name, area, Qmax, k, L2, n = calculateSewerValues(group,shapeType)
        
    pipe = {}
    pipe[SWWM_C.NAME] = name
    pipe[SWWM_C.AREA] = area
    pipe[VMAX] = Qmax/area
    pipe[K] = k
    pipe[TANK_INDEXES] = [*range(tankIndex,tankIndex+n,1)]
    pipe[SWWM_C.LEN] = L2
    
    return pipe, name, n

#Asumes RECT pipes have the same geom 2 than geom 1
def calculateSewerValues(group,shapeType):
    
    length = group[SWWM_C.LEN].sum()
    name = group.iloc[0,0] + " - " + group.iloc[-1,0]
    diam = group[SWWM_C.DIAM].min() #they are all the same
    slope = np.average(group[SLOPE], weights=group[SWWM_C.LEN])
    roughness = np.average(group[SWWM_C.ROUG], weights=group[SWWM_C.LEN])             
    Lc = 0.4 * diam / slope
    n1 = length/Lc
    n = round(n1) if n1 >= 1 else 1
    L2 = length/n
    
    c = (2*GRAVITY*diam*slope)**0.5
    a = 2.51*VISCO/(diam*c)
    b = roughness/(3.71*diam)

    if shapeType == SWWM_C.CIRC:
        area = math.pi * (diam**2)/4
    elif (shapeType == SWWM_C.REC) or (shapeType == SWWM_C.REC2):
        area = diam**2
    else:
        area = 0

    Qmax = area*(-2*math.log(a+b)*c) if slope > 0 else 0

    k = 0.64 * L2* (diam**2)/ Qmax
    
    return name, area, Qmax, k, L2, n

#Cathcments are named as the name the pipe section connected to 
#Converst flow and area to the values from SWMM to the units used in WEST
def createCatchmentWEST(name, element, timePatterns, isEnd=True):

    catchment = {}
    catchment[SWWM_C.NAME] = name
    
    #Attributes of catchments (equal to SWMM) ----------------------------
    catchment[SWWM_C.AREA] = element[SWWM_C.AREA] * 10000 #converts from ha to m2
    
    #Attributes from DWF----------------------------------------
    averageDWF = element[SWWM_C.INFLOW_MEAN]
    timePatternDWF = element[SWWM_C.INFLOW_PATTERNS]
    
    if (not math.isnan(averageDWF)) & (averageDWF != 0):
        npeople = convertMeanSWMMFlowToNPeopleWEST(averageDWF)
        tPatternP = timePatterns[timePatternDWF]
    else:
        npeople = None
        tPatternP = None
    
    catchment[N_PEOPLE] = npeople
    catchment[QPERPERSON_LABEL] = FLOW_PER_PERSON
    catchment[TIMEPATTERN] = tPatternP
    
    #Attributes from direct flows --------------------------------
    catchment[SWWM_C.DFLOW_BASELINE] = element[SWWM_C.DFLOW_BASELINE] * 86400 # convert it from m3/s to m3/d
    #directFlow[DDFLOW_TIMES] = timeSeries
    #directFlow[DFLOW_SFACTOR] = Sfactor
    
    #Aux attributes ---------------------------------------------
    catchment[END] = isEnd
    
    return catchment

#Converts the m3/s of SWMM to m3/d of WEST and divides per the flow per person constant to get the number of people
def convertMeanSWMMFlowToNPeopleWEST(averageDWF):

    averageM3d = averageDWF * 86400 #SWMM 'average' is in m3/s 
    npeople = math.ceil(averageM3d / FLOW_PER_PERSON) # flow per person is in m3/d as it is west

    return npeople

#Removes the first day of the ts as it considers the stabilisation period of the flow modeling
#With the rest of the time series, calculates the average flow per hour of the day and normalises it using the ts total average flow 
def convertTimeSeriesIntoDWF(ts):

    initialDate = ts.index[0]
    initialDateEvaluation = initialDate + timedelta(days=1) #it assumes that flow gets stable after one day!!
        
    dryWeather = ts[initialDateEvaluation:]
        
    totalMean = dryWeather.mean()
    hourly_pattern = dryWeather.groupby(dryWeather.index.hour).mean()
    
    assert list(hourly_pattern.index) == list(range(24)) #assumess that the TS is in order (starts in hour 0 and finish in 23)
        
    normalized_HP = hourly_pattern / totalMean
    NHP_stringList = list(map(str, normalized_HP))

    return NHP_stringList, totalMean

#It creates a catchment element in west with the timeseries of the input
def createInputWEST(name,input,tsInputs):

    inputWEST = {}

    inputWEST[SWWM_C.NAME] = name + "[input]"

    #Catchment attributes not used for representing an input -----------------
    inputWEST[SWWM_C.AREA] = 0
    inputWEST[SWWM_C.DFLOW_BASELINE] = 0
    
    #converts the time series into a pattern and number of people
    timeserie = tsInputs[input]
    tPatternP, averageDWF = convertTimeSeriesIntoDWF(timeserie)
    npeople = convertMeanSWMMFlowToNPeopleWEST(averageDWF)
    
    inputWEST[N_PEOPLE] = npeople
    inputWEST[TIMEPATTERN] = tPatternP
    
    
    #Aux attributes ------------------------------
    inputWEST[QPERPERSON_LABEL] = FLOW_PER_PERSON
    inputWEST[END] = True
    
    return inputWEST
    

#Each pipe section has the name as the initial and final node of the composing pipes 
# it has the mean of the slopes of the composing pipes, the mean (and only) diameter of the group, and the total length 
def getPathElements(dfs,elementsP,timePatterns,tSConnectedPoints):
    
    pipesSection = []
    catchments= []
    
    tankIndex = 1
    firstSection = True
    
    #Elements are in order from upstream to downstream
    elements = elementsP[PATH_ELEMENTS]
    initialElements = elementsP[PATH_ELEMENTS_INI]

    for df in dfs:
        #Classifies consecutive pipes with the same diameter
        groups = df.groupby((df[SWWM_C.DIAM] != df[SWWM_C.DIAM].shift()).cumsum())

        for i, groupDiameter in groups:
            
            shapes = groupDiameter.groupby(SWWM_C.SHAPE,sort=False)
            
            length = groupDiameter[SWWM_C.LEN].sum()
            if length == 0:
                print("Pump, Orifice or Weir")
                continue
            
            for shapeType, group  in shapes:
                
                #Creates and adds the pipe section to the list
                sewerSect, name, n = createSewerWEST(group,shapeType,tankIndex) 
                pipesSection.append(sewerSect)
                tankIndex += n
                
                #Creates and adds a catchment and/or and dwf to their list if they are connected at the beggining of the path
                if firstSection and (initialElements is not None):
                    catchment = createCatchmentWEST(name, initialElements,timePatterns,False)
                    catchments.append(catchment)
                    
                    firstSection = False
                
                #Creates and adds a catchment and/or a dwf to their list if they are connected to the end part of the sewer section
                connectingPipe= group.iloc[-1,0]
                if connectingPipe in elements.index:

                    #revisar que los datosno sean todos zero y si no crear catchment
                    element = elements.loc[connectingPipe].copy()
                    tsInput = element[MODELED_INPUT]

                    #If the ts is not empty it creates an input object
                    if tsInput is not None:
                        input = createInputWEST(name, tsInput,tSConnectedPoints)
                        catchments.append(input)

                    #if any of the others values are different from zero or null then it creates a catchment object
                    element.drop(labels=MODELED_INPUT,inplace=True)
                    element.fillna(0,inplace=True)
               
                    if ~((element == 0.0) | element.isna()).all():
                        catchment = createCatchmentWEST(name, element, timePatterns)
                        catchments.append(catchment)
            
                            
    return pipesSection, catchments

def convertListPathtoDF(path, links):
    
    #list of links names and information from the leaf node to the WTP
    linksPath= pd.DataFrame(path, columns= [SWWM_C.NAME])
    linksPath = linksPath.join(links,on= SWWM_C.NAME)
    
    return linksPath


def createLookPointsDF(nElements,pointsConnected,aggregating=AGG_CATCHMENT):
    
    #Get nodes out of the links with flow measures
    links = nElements[LINKS]
    stopNodes = links[links.index.isin(LINKS_MEASS_FLOW)][[SWWM_C.OUT_NODE]].copy()

    #gets created inputs (pipes that connect to the path with flow)
    createdInputs = pointsConnected.reset_index().rename(columns={SWWM_C.NAME:MODELED_INPUT})

    #gets output nodes of catchments, and aggregates by node. renames so that all stop nodes and cathcment nodes can be merged
    catchmNodes = nElements[SUBCATCHMENTS].groupby([SWWM_C.CATCH_OUT]).agg({SWWM_C.AREA: 'sum'}).reset_index().rename(
                                                    columns={'index':SWWM_C.OUT_NODE, SWWM_C.CATCH_OUT:SWWM_C.OUT_NODE})

    #selects the nodes where there is direct input, renames so that all stop nodes and cathcment nodes can be merged
    inputFlowNodes = nElements[DWFS].reset_index().rename(columns={SWWM_C.INFLOW_NODE:SWWM_C.OUT_NODE})
    directFlowNodes = nElements[DIRECTF].reset_index().rename(columns={SWWM_C.INFLOW_NODE:SWWM_C.OUT_NODE})
    
    print("Number of measurement points", len(stopNodes))
    print("Number of catchments nodes ",len(catchmNodes))
    print("Number of dw flows ", len(inputFlowNodes))
    print("Number of direct flows ",len(directFlowNodes))
    
    #Joins meassurement and catchment points
    #stopAndCatchment = pd.merge(stopNodes, catchmNodes, on=OUT_NODE, how='outer')
    stopAndInput = pd.merge(stopNodes, createdInputs, on=SWWM_C.OUT_NODE, how='outer')
    #Joins meassurement and catchment points
    #stopAndCatchmentAndInput = pd.merge(stopAndCatchment, createdInputs, on=OUT_NODE, how='outer')
    stopAndCatchmentAndInput = pd.merge(stopAndInput, catchmNodes, on=SWWM_C.OUT_NODE, how='outer')
    #Joins dry weather flows inputs points
    stopAndCatchmentDwf = pd.merge(stopAndCatchmentAndInput, inputFlowNodes, on=SWWM_C.OUT_NODE, how='outer')
    #Joins direct inputs points
    lookPoints = pd.merge(stopAndCatchmentDwf, directFlowNodes, on=SWWM_C.OUT_NODE, how='outer')
    
    #Gets the indexes to divide the path according to the level of aggregation
    if aggregating == AGG_CATCHMENT:
        #breakLinks = stopAndCatchmentAndInput
        breakLinks = stopAndInput
    else: #TODO other methods of aggregation ( This is no aggregation )
        breakLinks = None
    
    return lookPoints, breakLinks


def dividesPathByBreakPoints(linksPath,breakLinks):
    
    #Gets th index of the break points on the path and divides the pipe
    breakLinks = pd.merge(linksPath, breakLinks.set_index(SWWM_C.OUT_NODE), 
                              left_on=SWWM_C.OUT_NODE, right_index=True, how='inner',indicator=True).copy()
    indexBreakLinks = breakLinks.index + 1
    
    #separates the path in smaller parts according to the meassurable points
    dfs = np.split(linksPath, indexBreakLinks)    
    print("Number of divisions by meassurement or input nodes" , len(dfs))
    
    return dfs, indexBreakLinks

#checks if the initial node of the path is a outlet node of a catchment and if it is, it returns its area
def checkForInitialElements(dfPath,lookupLinks):
    
    initialNode = dfPath.iloc[0][SWWM_C.IN_NODE]
    lookupLinks.set_index(SWWM_C.OUT_NODE,inplace=True)
    initialElement = None
    
    if initialNode in lookupLinks.index:
        
        assert lookupLinks.loc[[initialNode]].shape[0] <=  1 ,f"There is more than one element with equal node out in the df"
        
        initialElement = {}
        initialElement[SWWM_C.OUT_NODE] = initialNode
        initialElement[SWWM_C.AREA] = lookupLinks.loc[initialNode,SWWM_C.AREA]
        initialElement[SWWM_C.INFLOW_MEAN] = lookupLinks.loc[initialNode,SWWM_C.INFLOW_MEAN]
        initialElement[SWWM_C.INFLOW_PATTERNS] = lookupLinks.loc[initialNode,SWWM_C.INFLOW_PATTERNS]
        initialElement[SWWM_C.DFLOW_BASELINE] = lookupLinks.loc[initialNode,SWWM_C.DFLOW_BASELINE]
        initialElement[MODELED_INPUT] = lookupLinks.loc[initialNode,MODELED_INPUT]
            
    return initialElement

def checkUniqueDWFPatterns(lookPointsPath):

    groupedPatterns = lookPointsPath.groupby(BREAK_POINT)[SWWM_C.INFLOW_PATTERNS].transform('nunique')
    assert len(lookPointsPath[BREAK_POINT][groupedPatterns > 1].unique()) == 0,f"Sewer sections with different DWF patterns are trying to be grouped."  
    assert lookPointsPath[SWWM_C.NAME].nunique() == lookPointsPath.shape[0],f"There are duplicate names"

# Joins the linkpath df to the lookpoints and slices the result df by element
# checks for elements at the initial point of the path 
# stores all the results in the proElements dictionary
def extractPathLookPoints(linksPath,lookPoints,breaklinksIndexPath):
    
    proElements = {}
    
    #selects the pipes IN THE PATH that are connected to look points
    lookPointsPath = linksPath.join(lookPoints.set_index(SWWM_C.OUT_NODE), on=SWWM_C.OUT_NODE).copy()
    
    #Finds the closest breaking point of each element --------------------------------------------------------------------------
    markerIndicesUnmodified = (breaklinksIndexPath - 1).tolist() #because previously one was added
    
    lookPointsPath[BREAK_POINT] = lookPointsPath.index.to_series().apply(lambda idx: min((n for n in markerIndicesUnmodified if n >= idx), default=None))
       
    # Check if there are no grouped sections with different dwf patterns --------------------------------------------------------
    checkUniqueDWFPatterns(lookPointsPath)

    # Groups by the sections elements to the nearest break point --------------------------------------------------------------------
    proElements[PATH_ELEMENTS] = lookPointsPath.groupby([BREAK_POINT]).agg({SWWM_C.NAME: 'last',SWWM_C.AREA:'sum', 
                                                                            SWWM_C.INFLOW_MEAN: 'sum', SWWM_C.INFLOW_PATTERNS: 'first',
                                                                            SWWM_C.DFLOW_BASELINE: 'sum', SWWM_C.DFLOW_SFACTOR : 'mean',
                                                                            MODELED_INPUT:'first'}).set_index([SWWM_C.NAME]) #TODO add other values
    
    #proElements[PATH_ELEMENTS].to_csv('02-Output/'+'elementsGrouped'+'.csv')
    lookPointsPath.to_csv('02-Output/'+'elementsImportantPath'+'.csv')
    
    #Checks for elements at the initial node of the path
    proElements[PATH_ELEMENTS_INI] = checkForInitialElements(linksPath,lookPoints)
    
    return proElements
    

def convertPathToSwrSectAndCatchts(linksPath, nElements,timeSeriesPointsCon, pointsConnected):
    
    #Calculate the slopes FOR OPTION IN THE INP = ELEVATION only for conduits
    linksPath[SLOPE] = (linksPath[SWWM_C.INOFF]-linksPath[SWWM_C.OUTOFF])/linksPath[SWWM_C.LEN]
    
    #linksPath.to_csv('02-Output/'+'elementspath'+'.csv')

    #Joins all important points (measured nodes, out nodes of catchments, input flows for dwf and direct flows) into a df
    lookPoints, breaklinks = createLookPointsDF(nElements,pointsConnected)
    
    #Divides the path in various sections using dfs
    pathDfs, indexbreakLinks = dividesPathByBreakPoints(linksPath,breaklinks)

    #converts the important points into pipes with the characteristics required
    proElements = extractPathLookPoints(linksPath,lookPoints,indexbreakLinks)
   
    #separates the path by diameter and converts the group of pipes into sewer sections, catchments, and dwf
    pipeSectionsProperties, catchmentsProperties = getPathElements(pathDfs,proElements,nElements[T_PATTERNS],timeSeriesPointsCon)
              
    print("Final number of pipe sections ", len(pipeSectionsProperties))
    return pipeSectionsProperties, catchmentsProperties

#find the pipes connected to the path giving it flow
def lookForOtherPipesConnected(linksPath,allLinks,fileOut):

    #Gets the links that are not in the path
    linksPathOutNodes = linksPath.set_index(SWWM_C.NAME)[SWWM_C.OUT_NODE]
    pipesNotPath = allLinks[~allLinks.index.isin(linksPathOutNodes.index)].copy()
    assert allLinks.shape[0] == pipesNotPath.shape[0] + linksPath.shape[0]
    
    #Gets links in the network connected to the path (but not part of it)
    pipesConnected = pipesNotPath[pipesNotPath[SWWM_C.OUT_NODE].isin(linksPathOutNodes)][[SWWM_C.OUT_NODE]].copy()
    pipesConnectedNames = pipesConnected.index.to_list()
    print("There are", len(pipesConnectedNames), "connections to the path")

    #Check that there are no more than one pipe per outnode!! TODO
    #-------TODO-------TODO-------TODO--------TODO-----TODO-----TODO

    #Gets the timeseries of the flow from the connections to the path
    tsDFNoZeros = None    
    if len(pipesConnectedNames) > 0:
        tsDF = None

        with Output(fileOut) as out:
            for pipe in pipesConnectedNames:
                
                ts = out.link_series(pipe, LinkAttribute.FLOW_RATE)
                
                if tsDF is None:
                    tsDF = pd.DataFrame.from_dict(ts, orient='index',columns=[pipe])
                else:
                    tsDF = tsDF.join(pd.DataFrame.from_dict(ts, orient='index',columns=[pipe]))
    
        # remove columns with only zeros
        tsDFNoZeros = tsDF.loc[:, (tsDF != 0).any(axis=0)]
        print(tsDF.shape[1] - tsDFNoZeros.shape[1],"connections to the path were removed due to no flow at anytime.") 

    #Removes from the list the pipes with only zero flow
    outNodesNoZeros = pipesConnected[pipesConnected.index.isin(tsDFNoZeros.columns.to_list())]
    
    return tsDFNoZeros, outNodesNoZeros


# Get pipe sections from detailed model ------------------------

In [4]:
file = 'DWF2022.inp' 

#Gets all the elements from the network used
nElements, outfile = getsNetwork(file)

#Get all paths from leaves to water treatment plant
pathsAll = fp.getPathToWTP(WTP_Tank,nElements[LINKS],nElements[LEAVES])

#---------------------------------------------------------------------------------
#Get St sacrament path as df
#Pipes in the path are in order from downstream to upstream
stSacrPathDF = convertListPathtoDF(pathsAll[ST_SACR_LEAF_NODE], nElements[LINKS])

#Gets the pipes connected to the path but not part of it with their flow timeseries
timeSeriesPointsCon, pointsConnected  = lookForOtherPipesConnected(stSacrPathDF,nElements[LINKS],outfile)
 
#converts the path from st sacrement to the WTP into sewer sections
pipesModel,catchmentsModel = convertPathToSwrSectAndCatchts(stSacrPathDF,nElements,timeSeriesPointsCon, pointsConnected) 

print("----------------------------")
print("Final number of catchments: ", len(catchmentsModel))
print("Final number of tanks in series:", pipesModel[-1][TANK_INDEXES][-1])

There were  17 subcatchments with area 0
Number of nodes  1493 , outlets  1421 , startPoints  72 , and after cleaned  70


C:\Users\lllov\AppData\Local\Temp\ipykernel_108364\3434437577.py:58: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  assert model.inp.options.loc['LINK_OFFSETS'][0] == SWWM_C.ELEVATION


R008120   pop from empty list
R571863   pop from empty list
There are 21 connections to the path
5 connections to the path were removed due to no flow at anytime.
Number of measurement points 5
Number of catchments nodes  118
Number of dw flows  91
Number of direct flows  133
Number of divisions by meassurement or input nodes 17
Pump, Orifice or Weir
Pump, Orifice or Weir
Pump, Orifice or Weir
Final number of pipe sections  24
----------------------------
Final number of catchments:  22
Final number of tanks in series: 46


c:\Users\lllov\OneDrive - Université Laval\Desktop\Lunita\02-PythonSWMM\SWMMtoWEST\westWorkEnv\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [13]:
stSacrPathDF

,Name,InletNode,...,Shape,Slope
0,DOM_7925,R007637,...,CIRCULAR,0.002000
1,DOM_56804,R007275,...,CIRCULAR,0.002090
2,DOM_12577,R007274,...,CIRCULAR,0.001958
3,DOM_56806,RA_11702,...,CIRCULAR,0.002003
4,DOM_12585,R007273,...,CIRCULAR,0.002002
...,...,...,...,...,...
110,DOM_35983,R014059,...,CIRCULAR,0.008524
111,U002_Pompe_IREU,RA_180174,...,NaN,NaN
112,DOM_1631422,RA_619971,...,CIRCULAR,0.009978
113,DOM_1640876,U89A,...,RECT_CLOSED,0.002442


In [5]:
catchmentsModel 

[{'Name': 'DOM_7925 - DOM_602606',
  'Area': 8880.0,
  'NumberPeople': 3068,
  'FlowPerPerson': 0.4,
  'TimePattern': ['0.35',
   '0.249',
   '0.174',
   '0.176',
   '0.178',
   '0.375',
   '1.137',
   '1.5',
   '1.731',
   '1.779',
   '1.627',
   '1.41',
   '1.245',
   '1.008',
   '0.889',
   '0.997',
   '1.285',
   '1.437',
   '1.406',
   '1.291',
   '1.207',
   '1.176',
   '0.847',
   '0.525'],
  'Baseline': 1719.3600000000001,
  'EndNode': False},
 {'Name': 'DOM_7925 - DOM_602606[input]',
  'Area': 0,
  'Baseline': 0,
  'NumberPeople': 16546,
  'TimePattern': ['0.8349720013081973',
   '0.8071398009804694',
   '0.7871290914266064',
   '0.785889500024899',
   '0.7864082550720318',
   '0.8332161512003997',
   '1.018823947857598',
   '1.1220453172287892',
   '1.1852048349064956',
   '1.201410445612298',
   '1.1659484729320828',
   '1.1110054632019908',
   '1.0672165342154196',
   '1.0072904672445222',
   '0.9738072940681994',
   '0.9968120554479352',
   '1.067670747010346',
   '1.11025

In [6]:
pipesModel

[{'Name': 'DOM_7925 - DOM_602606',
  'Area': 0.6361725123519332,
  'Vmax': 2.0576379880320053,
  'k': 74.69293176599832,
  'tanksIndexes': [1, 2, 3, 4],
  'Length': 188.60725},
 {'Name': 'UNI_602607 - UNI_8693',
  'Area': 0.6361725123519332,
  'Vmax': 1.9603801621410302,
  'k': 105.68062708472964,
  'tanksIndexes': [5],
  'Length': 254.241},
 {'Name': 'UNI_8674 - UNI_13606',
  'Area': 0.6361725123519332,
  'Vmax': 1.9123189258465227,
  'k': 87.19792453845383,
  'tanksIndexes': [6, 7, 8],
  'Length': 204.63333333333335},
 {'Name': 'UNI_13117 - UNI_13121',
  'Area': 0.2827433388230814,
  'Vmax': 4.2587699036007916,
  'k': 3.00037963606994,
  'tanksIndexes': [9, 10, 11, 12, 13, 14, 15, 16],
  'Length': 15.680874999999999},
 {'Name': 'UNI_1568728 - UNI_12850',
  'Area': 0.44178646691106466,
  'Vmax': 2.5323823250084474,
  'k': 29.533839696544106,
  'tanksIndexes': [17, 18, 19],
  'Length': 91.78233333333334},
 {'Name': 'UNI_1635183 - C21',
  'Area': 0.44178646691106466,
  'Vmax': 3.3396244

# Start modifying WEST MODEL --------------------------

In [7]:
#Variables to check that the models have

#Descriptions of the models 
SEWER = 'Sewer'
CATCHMENT = 'Catchment'
CONNECTOR = 'Connector_info'


In [8]:
#Variables that can be changed accordingly

#classes to be assigned to the west blocks by model
SEWER_CLASS = 'psvdmSewerTank_Classplus_decay'
CATCH_CLASS = 'Catchment_NoRetention_IndirectStormwater'
CONNECTOR_CLASS = 'Kosim1ToPSVDM_curve_tracerflow_ClassplusViral'


In [9]:

XML_MODEL_PROP_NAME = 'InstanceDisplayName'
XML_QUANTIY = 'Quantity'


XML_SEWER_NAMES =  'Sew_'
XML_CATCH_NAMES = 'Catchment_'
XML_CONN_NAMES = 'Connector_info_'

XML_AREA_SEWER = '.A'
XML_VMAX_SEWER = '.V_Max'
XML_K_SEWER = '.k'

XML_AREA_CATCH = '.TotalArea'

XML_CONN_VELCLASS = '.Vs_limit'
XML_CONN_VELMINCLASS = '.Vs_limit_min'

XML_DWF_NPEOPLE = '.Inhabitants' #Associated to a catchment in WEST
XML_DWF_CUSTOMFLOWPATTERN = '.DWF.CustomFlow(H' #Associated to a catchment in WEST

#Other parameter modified in the OLD WEST project
XML_DWF_QPERPERSON = '.WastewaterPerIE' #Associated to a catchment in WEST
XML_DWF_Q_INDUSTRY = '.Q_Industry'

XML_DWF_INFILTRATION = '.Infiltration' #Associated to a catchment in WEST | it is repeated directly to the catchment i.e., .Infiltration 
XML_WRUNOFF_MAXWETTINGLOSSES = '.FRunoff.MaxWettingLosses'#Associated to a catchment in WEST | it is repeated directly to the catchment i.e., .MaxWettingLosses 
XML_CATH_YEARLYEVAPO = '.YearlyEvaporation'  
XML_CATH_ACCMAXAREAL = '.M_acc_max_areal'


#NOT BEING USED YET-------------------------------------------------------------------------------------------

XML_ISTROM_AREA = '.IndirectStorm.A'
XML_ISTROM_QMAX = '.IndirectStorm.Q_Max'
XML_ISTROM_VMAX ='.IndirectStorm.V_Max'
XML_ISTROM_KOUT = '.IndirectStorm.k_out'

XML_SEWER_DRATE = '.D_rateSARS'
XML_SEWER_KSAT = '.K_sat'
XML_SEWER_RESUSPENK = '.k_resuspension'  
XML_SEWER_RESUSPENN = '.n_resuspension' 


#Also not used in the current WEST model --------------------------------------------------------------------
# FRunoff.PerviousFraction


#Calibration parameters



In [10]:
INFILTRATION = 'Infiltracion'


#Variables and their translation 
#CATCH_DICT_SET = [AREA,N_PEOPLE,QPERPERSON_LABEL,INFILTRATION]
#CATCH_XML_SET = [XML_AREA_CATCH,XML_DWF_NPEOPLE,XML_DWF_QPERPERSON,XML_DWF_INFILTRATION]

CATCH_DICT_SET = [SWWM_C.AREA,N_PEOPLE,QPERPERSON_LABEL,SWWM_C.DFLOW_BASELINE]
CATCH_XML_SET = [XML_AREA_CATCH,XML_DWF_NPEOPLE,XML_DWF_QPERPERSON,XML_DWF_Q_INDUSTRY]

In [11]:
def getQuantityXML(qName, qValue):
    
    # Create a new Quantity element and its sub-elements
    new_quantity = ET.Element(XML_QUANTIY, attrib={'Name': qName})
    props = ET.SubElement(new_quantity, 'Props')
    prop = ET.SubElement(props, 'Prop', attrib={'Name': 'Value', 'Value': qValue})
    
    return new_quantity

#sets the class of the model and obtains the i (name) of its type
def getNameAndSetClass(submodel,classSub,typeNameSub):
    
    submodel.find("./Props/Prop[@Name='ClassName']").set('Value', classSub) # Always the same class
    instName = submodel.find("./Props/Prop[@Name='InstanceName']").get('Value')
            
    try:
        iType = int(re.split("^" + typeNameSub, instName)[1])

    except Exception as e:
        raise Exception("The InstanceName of the sewer (XML) does not follow the expected naming convention (" + str(e) + ")")
        
    return iType, instName

def createsOrModifyQuantity(property,instName,XMLval,quant_XMLelem,root):

    #Get the values to modify
    val = str(property) 
    valQuantityName = '.' + instName + XMLval
    valQuantity = root.find(".//Quantities/Quantity[@Name='"+ valQuantityName +"']")
                
    #if exists changes the value, if not then creates the XML
    if (valQuantity is None):

        valXML = getQuantityXML(valQuantityName, val)
        quant_XMLelem.append(valXML)
                    
    else:
        valQuantity.find("./Props/Prop[@Name='Value']").set('Value', val)

    return quant_XMLelem,root

def modifySewerModel(root, submodel, props):
    
    iSewer,instName = getNameAndSetClass(submodel,SEWER_CLASS,XML_SEWER_NAMES)

    #Iterates the sewer section properties to find the required one and allocate
    for sectP in props:

        # if the sewer section properties apply to the current index!!! (IMPORTANT) 
        if iSewer in sectP[TANK_INDEXES]: 
            
            #Sets the name
            nameSewer=  sectP[SWWM_C.NAME] + "("+ str(iSewer - min(sectP[TANK_INDEXES])) + ")"
            submodel.find("./Props/Prop[@Name='InstanceDisplayName']").set('Value', nameSewer) 
            quantities_elem = root.find('.//Quantities')

            for P,XMLval in zip([SWWM_C.AREA,VMAX,K],[XML_AREA_SEWER,XML_VMAX_SEWER,XML_K_SEWER]):
                
                val = sectP[P]

                if ((val is not None) & (val != 0)):
                    quantities_elem,root = createsOrModifyQuantity(val,instName,XMLval,quantities_elem,root) 
                
    return root


#the number of people was calculated using the DWFs of the SWMM model
def modifyCatchmentModel(root, submodel, props):
    
    iCatch,instName = getNameAndSetClass(submodel,CATCH_CLASS,XML_CATCH_NAMES)
    i = 1
    
    #Iterates the catchment properties to find the required one and allocate
    for catchM in props:
        
        if iCatch == i:
            
            #Sets the name
            pos = "(After)" if catchM[END] == True else "(Before)"
            name = catchM[SWWM_C.NAME] + pos
            submodel.find("./Props/Prop[@Name='InstanceDisplayName']").set('Value', name) 

            quantities_elem = root.find('.//Quantities')

            #For simple values
            for P,XMLval in zip(CATCH_DICT_SET,CATCH_XML_SET):

                val = catchM[P]

                if ((val is not None) & (val != 0)):
                    quantities_elem,root = createsOrModifyQuantity(val,instName,XMLval,quantities_elem,root)
                elif ((val is None) | (val == 0))  & ((P == SWWM_C.AREA) | (P == N_PEOPLE)): #If no values are given the 
                    val = 0 #Replace with zero in cas is None
                    quantities_elem,root = createsOrModifyQuantity(val,instName,XMLval,quantities_elem,root)
            
            #For the time pattern
            if (catchM[TIMEPATTERN] is not None):
                for h,vh in zip([f"{hour:02})" for hour in range(0,24)],catchM[TIMEPATTERN]):

                    quantities_elem,root = createsOrModifyQuantity(vh,instName,XML_DWF_CUSTOMFLOWPATTERN + h,quantities_elem,root)
            
        i+=1
            
    return root

def modifyConnectorModel(root, submodel,velTSS,velMin):
    
    nClasses = 10
    
    iConnector,instName = getNameAndSetClass(submodel,CONNECTOR_CLASS,XML_CONN_NAMES)
    
    #if exists changes the value, if not then creates the XML
    #Assumes if one of the properties exist the others do too
    velMinName = '.' + instName + XML_CONN_VELMINCLASS 
    velMinVal = root.find(".//Quantities/Quantity[@Name='"+ velMinName +"']")
    if (velMinVal is None):
        
        quantities_elem = root.find('.//Quantities')
        
        #Create the xml tags using as dictionaries
        velMinXML = getQuantityXML(velMinName, str(velMin))
        quantities_elem.append(velMinXML)
        
        for vi, vel in zip(range(0,nClasses,1),velTSS):
            
            #Create the xml tags using as dictionaries
            velName = '.' + instName + XML_CONN_VELCLASS + '(C'+ str(vi) +')'
            velXML = getQuantityXML(velName, str(vel)) 
            quantities_elem.append(velXML)
                                     
    else:
        velMinVal.find("./Props/Prop[@Name='Value']").set('Value', str(velMin))
        
        for vi, vel in zip(range(0,nClasses,1),velTSS):
            
            velName = '.' + instName + XML_CONN_VELCLASS + '(C'+ str(vi) +')'
            velVal= root.find(".//Quantities/Quantity[@Name='"+ velName +"']")
            velVal.find("./Props/Prop[@Name='Value']").set('Value', str(vel))
   
    return root

#Props must be in order
def setSewerSectionsVals(xml,xmlOut,propsSewer,propsCath,connectorProps):

    # Read the XML file
    tree = ET.parse(xml)
    root = tree.getroot()  
    submodels = root.findall('.//SubModel') 
    
    print("Number of submodels found ",len(submodels))
     
    #count of sewers and catchments models found in the XML 
    nSewers, nCatchments, nConnectors = 0,0,0
    
    # Iterate over the SubModels
    for submodel  in submodels:
                
        #if the model is a sewer
        if submodel.find("./Props/Prop[@Name='Desc']").get('Value') ==  SEWER: 
            root= modifySewerModel(root, submodel, propsSewer)
            nSewers += 1
        elif submodel.find("./Props/Prop[@Name='Desc']").get('Value') ==  CATCHMENT:
            root= modifyCatchmentModel(root, submodel, propsCath)
            nCatchments += 1
        elif submodel.find("./Props/Prop[@Name='Desc']").get('Value') ==  CONNECTOR:
            root= modifyConnectorModel(root, submodel,connectorProps[1],connectorProps[0])
            nConnectors += 1
           
    print("The number of sewers found were ", nSewers, ", catchments ", nCatchments, " and connectors ", nConnectors)
    
    # Save the modified XML to a new file
    ET.indent(tree, space="\t", level=0)
    tree.write(xmlOut)
 
    


In [12]:
connProps = []
connProps.append(18)
connProps.append([23.6,47.2,65.8,141.5,283,565.9,800,943.2,1800,2829.6])

#the last section , the propertie tank index, the last index props[-1][TANK_INDEXES][-1]
setSewerSectionsVals('01-Data/05-NewWESTModel46Sections/Project1.Layout.xml',
                     '02-Output/03-NewWEST46Sections/Project1Mod.Layout.xml',
                     pipesModel,catchmentsModel,connProps)




FileNotFoundError: [Errno 2] No such file or directory: '01-Data/05-NewWESTModel46Sections/Project1.Layout.xml'

: 